In [21]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

In [22]:
X_train_res = pd.read_csv("/Users/sanketsaxena/Desktop/consumerComplaint/Artifacts/X_train_res.csv")
X_test_res = pd.read_csv("/Users/sanketsaxena/Desktop/consumerComplaint/Artifacts/X_test_res.csv")
y_train_res = pd.read_csv("/Users/sanketsaxena/Desktop/consumerComplaint/Artifacts/y_train_res.csv")
y_test_res = pd.read_csv("/Users/sanketsaxena/Desktop/consumerComplaint/Artifacts/y_test_res.csv")

In [28]:
train_data = pd.DataFrame({
    'text': X_train_res['text'],
    'label': y_train_res['label']
})

In [29]:
sampling_strategy = {
    'Credit Reporting/Repair Services/Consumer Reports': 30000,
    'Debt Collection': len(train_data[train_data['label'] == 'Debt Collection'])//10,
    'Credit/Prepaid Cards': len(train_data[train_data['label'] == 'Credit/Prepaid Cards'])//10,
    'Mortgage': len(train_data[train_data['label'] == "Mortgage"])//10,
    'Bank Account Services': len(train_data[train_data['label'] == 'Bank Account Services'])//10,
    'Student loan': len(train_data[train_data['label'] == 'Student loan'])//10,
    'Money Services/Transfer': len(train_data[train_data['label'] == 'Money Services/Transfer'])//10,
    'Personal Loan': len(train_data[train_data['label'] == 'Personal Loan'])//10,
    'Vehicle loan or lease': len(train_data[train_data['label'] == 'Vehicle loan or lease'])//10,
    'Other financial service':len(train_data[train_data['label'] == 'Other financial service'])//10
}

In [30]:
rus = RandomUnderSampler(sampling_strategy=sampling_strategy)

# Separate features and labels
X_train_combined = train_data[['text']]
y_train_combined = train_data['label']

# Apply RandomUnderSampler
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_combined, y_train_combined)

In [31]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder

In [32]:
test_data = pd.DataFrame({
    'text': X_test_res['text'],
    'label': y_test_res['label']
})

In [33]:
sampling_strategy = {
    'Credit Reporting/Repair Services/Consumer Reports': 3000,
    'Debt Collection': len(test_data[test_data['label'] == 'Debt Collection'])//10,
    'Credit/Prepaid Cards': len(test_data[test_data['label'] == 'Credit/Prepaid Cards'])//10,
    'Mortgage': len(test_data[test_data['label'] == "Mortgage"])//10,
    'Bank Account Services': len(test_data[test_data['label'] == 'Bank Account Services'])//10,
    'Student loan': len(test_data[test_data['label'] == 'Student loan'])//10,
    'Money Services/Transfer': len(test_data[test_data['label'] == 'Money Services/Transfer'])//10,
    'Personal Loan': len(test_data[test_data['label'] == 'Personal Loan'])//10,
    'Vehicle loan or lease': len(test_data[test_data['label'] == 'Vehicle loan or lease'])//10,
    'Other financial service':len(test_data[test_data['label'] == 'Other financial service'])//10
}

In [34]:
rus = RandomUnderSampler(sampling_strategy=sampling_strategy)

# Separate features and labels
X_test_combined = test_data[['text']]
y_test_combined = test_data['label']

# Apply RandomUnderSampler
X_test_resampled, y_test_resampled = rus.fit_resample(X_test_combined, y_test_combined)

In [35]:
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

In [11]:
train_encodings= tokenizer(X_train_resampled['text'].tolist(), return_tensors='tf', max_length=512, padding='max_length', truncation=True)


In [12]:
import pickle

# Save the tokenized data to a pickle file
with open('/Users/sanketsaxena/Desktop/consumerComplaint/Artifacts/train_encodings_tokenized.pkl', 'wb') as f:
    pickle.dump(train_encodings, f)

In [13]:
test_encodings= tokenizer(X_test_resampled['text'].tolist(), return_tensors='tf', max_length=512, padding='max_length', truncation=True)


In [14]:
import pickle

# Save the tokenized data to a pickle file
with open('/Users/sanketsaxena/Desktop/consumerComplaint/Artifacts/test_encodings_tokenized.pkl', 'wb') as f:
    pickle.dump(test_encodings, f)

In [36]:
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_resampled.values)
y_test_encoded = le.transform(y_test_resampled.values)

In [37]:
type(y_test_encoded)

numpy.ndarray

In [16]:
train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask']
}, y_train_encoded))

In [17]:
test_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask']
}, y_test_encoded))

In [18]:
tf.data.experimental.save(train_dataset, '/Users/sanketsaxena/Desktop/consumerComplaint/Artifacts/tensorflowDatasetTrain')

Instructions for updating:
Use `tf.data.Dataset.save(...)` instead.


In [19]:
tf.data.experimental.save(test_dataset, '/Users/sanketsaxena/Desktop/consumerComplaint/Artifacts/tensorflowDatasetTest')

In [20]:
train_dataset = train_dataset.shuffle(len(y_train_encoded)).batch(32)
test_dataset = test_dataset.batch(32)

In [22]:
len(set(y_train_encoded))

10

In [23]:
# Load the pre-trained model
model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(set(y_train_encoded)))

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [25]:


# Compile the model
optimizer = tf.keras.optimizers.legacy.Adam()
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=2, validation_data=test_dataset, verbose=1)


Epoch 1/2
   5/2896 [..............................] - ETA: 33:54:56 - loss: 7.9630 - accuracy: 0.1187